In [1]:
import nltk
import pandas as pd
from nltk.stem import WordNetLemmatizer 
lemmatizer = WordNetLemmatizer()

In [15]:
df = pd.read_csv(r"c:\Users\User\Downloads\European Restaurant Reviews.csv")
df.head()

,Country,Restaurant Name,Sentiment,Review Title,Review Date,Review
0,France,The Frog at Bercy Village,Negative,Rude manager,May 2024 •,The manager became agressive when I said the c...
1,France,The Frog at Bercy Village,Negative,A big disappointment,Feb 2024 •,"I ordered a beef fillet ask to be done medium,..."
2,France,The Frog at Bercy Village,Negative,Pretty Place with Bland Food,Nov 2023 •,"This is an attractive venue with welcoming, al..."
3,France,The Frog at Bercy Village,Negative,Great service and wine but inedible food,Mar 2023 •,Sadly I used the high TripAdvisor rating too ...
4,France,The Frog at Bercy Village,Negative,Avoid- Worst meal in Rome - possibly ever,Nov 2022 •,From the start this meal was bad- especially g...


In [16]:
df.iloc[0]['Review']

'The manager became agressive when I said the carbonara was not good. She was rude. It was  a very bad experience. The worst in Rome in many years.'

In [17]:
import re
import nltk
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords

# Download stopwords (run once)
nltk.download('stopwords')

ps = PorterStemmer()
stop_words = set(stopwords.words('english'))

corpus = []

for i in range(len(df)):
    review = re.sub('[^a-zA-Z]', ' ', df['Review'][i])
    review = review.lower()
    review = review.split()
    review = [ps.stem(word) for word in review if word not in stop_words]
    review = ' '.join(review)
    corpus.append(review)

print(corpus)


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


['manag becam agress said carbonara good rude bad experi worst rome mani year', 'order beef fillet ask done medium got well done cook dri told took steak minut brought anoth steak complet rare left steak ask charg fool price', 'attract venu welcom albeit somewhat slow servic offer pleasingli present everyth tast though ingredi assembl without season love restaur except food', 'sadli use high tripadvisor rate liter fortun eat number fine restaur know amaz buzz mouth fine freshli cook food great chef never eaten tast menu never ad hoc great ambienc attent friendli servic pair wine except mayb hit night chef everi singl cours food certainli freshli cook pass kitchen way bathroom notic plate precook food readi go tast menu plate odd dish disappoint would accept might palat like everi plate food hot bland lack spark probabl sit around l serv staff great wine fail finish plate offer tast menu look good valu simpl bowl hot freshli cook pasta would done better justic wine ask admit great engli

In [18]:
corpus[0]

'manag becam agress said carbonara good rude bad experi worst rome mani year'

In [19]:
df.isnull().sum()

Country            0
Restaurant Name    0
Sentiment          0
Review Title       0
Review Date        0
Review             0
dtype: int64

In [29]:
X = df['Review']
y = df['Sentiment']

In [30]:
y.value_counts(1)*100

Sentiment
Positive    82.356858
Negative    17.643142
Name: proportion, dtype: float64

In [31]:
# Train Test Split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.30, random_state = 0)

In [32]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
X_train_vect = vectorizer.fit_transform(X_train)
X_test_vect = vectorizer.transform(X_test)

In [33]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(solver='lbfgs')
clf.fit(X_train_vect,y_train)

,"penalty penalty: {'l1', 'l2', 'elasticnet', None}, default='l2'Specify the norm of the penalty:- `None`: no penalty is added;- `'l2'`: add a L2 penalty term and it is the default choice;- `'l1'`: add a L1 penalty term;- `'elasticnet'`: both L1 and L2 penalty terms are added... warning:: Some penalties may not work with some solvers. See the parameter `solver` below, to know the compatibility between the penalty and solver... versionadded:: 0.19 l1 penalty with SAGA solver (allowing 'multinomial' + L1).. deprecated:: 1.8 `penalty` was deprecated in version 1.8 and will be removed in 1.10. Use `l1_ratio` instead. `l1_ratio=0` for `penalty='l2'`, `l1_ratio=1` for `penalty='l1'` and `l1_ratio` set to any float between 0 and 1 for `'penalty='elasticnet'`.",'deprecated'
,"C C: float, default=1.0Inverse of regularization strength; must be a positive float.Like in support vector machines, smaller values specify strongerregularization. `C=np.inf` results in unpenalized logistic regression.For a visual example on the effect of tuning the `C` parameterwith an L1 penalty, see::ref:`sphx_glr_auto_examples_linear_model_plot_logistic_path.py`.",1.0
,"l1_ratio l1_ratio: float, default=0.0The Elastic-Net mixing parameter, with `0 <= l1_ratio <= 1`. Setting`l1_ratio=1` gives a pure L1-penalty, setting `l1_ratio=0` a pure L2-penalty.Any value between 0 and 1 gives an Elastic-Net penalty of the form`l1_ratio * L1 + (1 - l1_ratio) * L2`... warning:: Certain values of `l1_ratio`, i.e. some penalties, may not work with some solvers. See the parameter `solver` below, to know the compatibility between the penalty and solver... versionchanged:: 1.8 Default value changed from None to 0.0... deprecated:: 1.8 `None` is deprecated and will be removed in version 1.10. Always use `l1_ratio` to specify the penalty type.",0.0
,"dual dual: bool, default=FalseDual (constrained) or primal (regularized, see also:ref:`this equation `) formulation. Dual formulationis only implemented for l2 penalty with liblinear solver. Prefer `dual=False`when n_samples > n_features.",False
,"tol tol: float, default=1e-4Tolerance for stopping criteria.",0.0001
,"fit_intercept fit_intercept: bool, default=TrueSpecifies if a constant (a.k.a. bias or intercept) should beadded to the decision function.",True
,"intercept_scaling intercept_scaling: float, default=1Useful only when the solver `liblinear` is usedand `self.fit_intercept` is set to `True`. In this case, `x` becomes`[x, self.intercept_scaling]`,i.e. a ""synthetic"" feature with constant value equal to`intercept_scaling` is appended to the instance vector.The intercept becomes``intercept_scaling * synthetic_feature_weight``... note:: The synthetic feature weight is subject to L1 or L2 regularization as all other features. To lessen the effect of regularization on synthetic feature weight (and therefore on the intercept) `intercept_scaling` has to be increased.",1
,"class_weight class_weight: dict or 'balanced', default=NoneWeights associated with classes in the form ``{class_label: weight}``.If not given, all classes are supposed to have weight one.The ""balanced"" mode uses the values of y to automatically adjustweights inversely proportional to class frequencies in the input dataas ``n_samples / (n_classes * np.bincount(y))``.Note that these weights will be multiplied with sample_weight (passedthrough the fit method) if sample_weight is specified... versionadded:: 0.17 *class_weight='balanced'*",None
,"random_state random_state: int, RandomState instance, default=NoneUsed when ``solver`` == 'sag', 'saga' or 'liblinear' to shuffle thedata. See :term:`Glossary ` for details.",None
,"solver solver: {'lbfgs', 'liblinear', 'newton-cg', 'newton-cholesky', 'sag', 'saga'}, default='lbfgs'Algorithm to use in the optimization problem. Default is 'lbfgs'.To choose a solver, you might want to consider the following aspects:- 'lbfgs' is a good default solver because it works reasonably well for a wide class of problems.- For :term:`mul

In [34]:
from sklearn.metrics import accuracy_score, classification_report
y_pred = clf.predict(X_test_vect)
accuracy_score(y_test,y_pred)
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

    Negative       1.00      0.59      0.75        79
    Positive       0.92      1.00      0.96       372

    accuracy                           0.93       451
   macro avg       0.96      0.80      0.85       451
weighted avg       0.93      0.93      0.92       451



In [ ]:
from textblob import TextBlob
import re

def predict_sentiment(text):
    # Correct spelling
    text = str(TextBlob(text).correct())
    # Remove non-letter characters and lowercase
    text = re.sub('[^a-zA-Z]', ' ', text).lower()
    
    # Analyze sentiment using TextBlob
    sentiment = TextBlob(text).sentiment.polarity  # returns a value between -1 and 1
    
    if sentiment > 0:
        return "Positive Review"
    elif sentiment < 0:
        return "Negative Review"
    else:
        return "Neutral Review"

# Example usage
reviews = [
    "this resturnt is not good",
    "the food was terrible",
    "I love this place",
    "I hate this service"
]

for review in reviews:
    print(f"Review: {review} --> {predict_sentiment(review)}")


Review: this resturnt is not good --> None
Review: the food was terrible --> None
Review: I love this place --> None
Review: I hate this service --> None
